In [11]:
import mysql.connector as sql
import pandas as pd
from pandas import DataFrame
import math
import numpy as np

connection = sql.connect(
    host = "hh-r-sql-1",
    user = "ppatel",
    passwd = "P4t3lP@!"   
)
crsr = connection.cursor()

# Defined DataFrames
#df = 'Activations for each retailer on average of last 12 weeks
#df1 = Stock at Warehouse on previous day(Yesterday)
#df2 = Calculation of Sell Through on current date
#df3 = Stock at Retailer on previous day(Yesterday)

# Getting Activations for all retailers for each games in last 12 weeks(SQL)
Activations = """SELECT b.ret_cd,
       d.trading_name,
       c.product_id,
       c.product_name,
       ROUND((bid/4),2)    Avg_Activation,
       Max(bid)              Max_Activation
FROM   (SELECT Count(book_id) AS bid,
               ret_cd,
               product_id
        FROM   pdb.snw_activations
        WHERE  activation_date BETWEEN Date_sub(Curdate(), INTERVAL 12 WEEK) AND Curdate()
        GROUP  BY ret_cd,
                  product_id) a
       LEFT JOIN pdb.retailer_outlet b
              ON a.ret_cd = b.ret_cd
       LEFT JOIN pdb.snw_master c
              ON a.product_id = c.product_id
       LEFT JOIN pdb.retailer_outlet d
              ON a.ret_cd = d.ret_cd
WHERE  d.status = 1
GROUP  BY b.ret_cd,
          d.trading_name,
          c.product_id,
          c.product_name,
          a.bid"""

crsr.execute(Activations)
df = DataFrame(crsr.fetchall())
df.columns = ["Retailer_id","Retailer_Name","Product_id","Product_Name","Avg_Activation","Max_Activation"] #Creating columns Names
df['Avg_Activation'] = df['Avg_Activation'].astype('float')

# Getting Stock level for each game at Warehouse(SQL)
Stock = """SELECT product_id,
       stock_level
FROM   pdb.snw_master_stock_level a
       LEFT JOIN pdb.v_business_day b ON a.bd_id = b.bd_id
WHERE  bd_date = Curdate() - 1
       AND stock_level > 100"""

crsr.execute(Stock)
df1 = DataFrame(crsr.fetchall())
df1.columns = ["Product_id","stock_level"] #Creating columns Names

#Calculating Sell Through for each game(SQL)
Sell_Through = """SELECT a.product_id,
       a.product_name,
       a.activation_start_date,
       Round(( a.ticket_price / 100 ) * a.tickets_in_game, 0) AS print_run,
       Sum(Round(( ( a.ticket_price / 100 ) * a.tickets_in_book ), 0)) AS Total_book_revenue,
       Round((ticket_price / 100),0) AS Price_Point
FROM   pdb.snw_master a
       LEFT JOIN pdb.snw_activations b ON a.product_id = b.product_id
WHERE  a.prod_sts = 2
       AND activation_start_date BETWEEN Date_sub(Curdate(), INTERVAL 1.5 year) AND Curdate()
GROUP  BY product_id,
          product_name,
          activation_start_date"""

crsr.execute(Sell_Through)
df2 = DataFrame(crsr.fetchall())
df2.columns = ["Product_id","Game_Name","activation_start_date","Print_Run","Sell_Through","Price_Point"] #Creating columns Names
df2['Print_Run'] = df2['Print_Run'].astype(int) 
df2['Sell_Through'] = df2['Sell_Through'].astype(int)
df2['Price_Point'] = df2['Price_Point'].astype(int)
#Calculation of Sales Through
df2['Sell_Through(%)'] = round(df2['Sell_Through'] * 100 / df2['Print_Run'],2)

#Getting Available stock at each retailer for individual game(SQL)
Available = """SELECT s1.ret_cd,
       s3.trading_name,
       s1.product_id,
       s4.product_name,
       s2.Total_books,
       s1.Available_date
FROM   (SELECT ret_cd,
               product_id,
               Max(available_date) AS Available_date
        FROM   pdb.snw_available
        GROUP  BY product_id,
                  ret_cd) AS s1
       INNER JOIN (SELECT a.product_id,
                          a.ret_cd,
                          Count(a.book_id) AS Total_books
                   FROM   pdb.snw_available AS a
                          INNER JOIN (SELECT ret_cd,
                                             product_id,
                                             Max(available_date) AS
                                             Available_date
                                      FROM   pdb.snw_available
                                      GROUP  BY product_id,
                                                ret_cd) AS s1
                                  ON a.product_id = s1.product_id
                                     AND a.ret_cd = s1.ret_cd
                   WHERE  a.available_date = s1.Available_date
                   GROUP  BY a.product_id,
                             a.ret_cd) AS s2
               ON s1.product_id = s2.product_id
                  AND s1.ret_cd = s2.ret_cd
       LEFT JOIN pdb.snw_master AS s4
              ON s4.product_id = s1.product_id
       LEFT JOIN pdb.retailer_outlet s3
              ON s3.ret_cd = s1.ret_cd
WHERE  prod_sts = 2
       AND activation_end_date > Curdate()
       AND s3.ret_cd IS NOT NULL
       AND s3.status = 1"""

crsr.execute(Available)
df3_1 = DataFrame(crsr.fetchall())
df3_1.columns = ["Retailer_id","Retailer_Name","Product_id","Product_Name","Stock_Available","Available_date"] #Creating columns Names


Available = """SELECT s1.ret_cd,
       s3.trading_name,
       s1.product_id,
       s4.product_name,
       s2.book_count,
       s1.max_available_date
FROM   (SELECT ret_cd,
               product_id,
               Max(available_date) AS max_available_date
        FROM   pdb.snw_available
        GROUP  BY product_id,
                  ret_cd) AS s1
       INNER JOIN (SELECT a.product_id,
                          a.ret_cd,
                          Count(a.book_id) AS book_count
                   FROM   pdb.snw_activations AS a
                          INNER JOIN (SELECT ret_cd,
                                             product_id,
                                             Max(available_date) AS
                                             max_available_date
                                      FROM   pdb.snw_available
                                      GROUP  BY product_id,
                                                ret_cd) AS s1
                                  ON a.product_id = s1.product_id
                                     AND a.ret_cd = s1.ret_cd
                   WHERE  a.activation_date > s1.max_available_date
                   GROUP  BY a.product_id,
                             a.ret_cd) AS s2
               ON s1.product_id = s2.product_id
                  AND s1.ret_cd = s2.ret_cd
       LEFT JOIN pdb.snw_master AS s4
              ON s4.product_id = s1.product_id
       LEFT JOIN pdb.retailer_outlet s3
              ON s3.ret_cd = s1.ret_cd
WHERE  prod_sts = 2
       AND activation_end_date > Curdate()
       AND s3.ret_cd IS NOT NULL
       AND s3.status = 1
       AND s1.max_available_date > Date_sub(Curdate(), INTERVAL 8 week) """

crsr.execute(Available)
df3_2 = DataFrame(crsr.fetchall())
df3_2.columns = ["Retailer_id","Retailer_Name","Product_id","Product_Name","Stock_Activated","Max_Available_date"] #Creating columns Names


df3 = pd.merge(df3_1,df3_2,'left', on = ['Retailer_id','Product_id'])

df3['Available_books'] = df3['Stock_Available'] - df3['Stock_Activated']

df3 = df3[["Retailer_id","Retailer_Name_x","Product_id","Product_Name_x","Available_books"]]


# Merging all dataframes into the final dataframe 
report1 = pd.merge(df, df2,'left', on = ['Product_id'])
report2 = pd.merge(report1,df1,'left', on =['Product_id'])
report0 = pd.merge(report2,df3,'left', on = ['Retailer_id','Product_id'])

# Rounding up the allocated stock if sell through is greater than 60 and rounding down if less than 60.Can be discussed 
report0["Allocated"] = np.where(report0["Sell_Through(%)"] > 60, report0["Avg_Activation"].apply(np.floor), report0["Avg_Activation"].apply(np.ceil))

# Removal of Games with 0 stock level. This will make sure we can allocate the books untill stock gets zero.
report00 = report0[report0["stock_level"] > 0]

# Final dataframe to work with
report = report00[["Retailer_id","Retailer_Name_x","Product_id","Product_Name_x","Avg_Activation","Sell_Through(%)","Allocated","Available_books","Price_Point"]]

# Sort the data in the final data frame by requested stock in descending order
report = report.sort_values('Allocated', ascending = False)

# Allocate the stock to each retailer
for index, row in report.iterrows():
    requested_stock = row['Allocated']
    retailer = row['Retailer_id']
    product = row['Product_id']
    
    # Retrieving the available stock for the product at the warehouse
    available_stock = df1.loc[df1['Product_id'] == product,'stock_level'].iloc[0]
    
    # Calculate the difference between the requested(Calculative Allocation) stock and the available stock
    difference = available_stock - requested_stock
    
    if difference > 0:
        # Allocate the requested stock to each retailer
        report.loc[(report['Retailer_id'] == retailer) & (report['Product_id'] == product),'Allocated'] = requested_stock
        # update the stock level after each allocation
        df1.loc[df1['Product_id'] == product,'stock_level'] -= requested_stock
    else:
        # Allocated the available stock(at Warehouse) to each retailer if the stock is less than calculated allocations
        report.loc[(report['Retailer_id'] == retailer) & (report['Product_id'] == product),'Allocated'] = available_stock
        # set the available stock at the ware house to zero
        df1.loc[df1['Product_id'] == product,'stock_level'] = 0

# Sort the final report by retailer id
report = report.sort_values('Retailer_id', ascending = True)



# Initialize a dictionary to store the allocation data 

allocation_data = {}

# Iterate through each row in the data frame
for index, row in report.iterrows():
    retailer = row['Retailer_id']
    Price = row['Price_Point']
    allocated = row['Allocated']
    Sell_through = row['Sell_Through(%)']
    
    # Checking if the retailer is already in the dictionary
    if retailer_id not in allocation_data:
        # If not in the dictionary ,add it and set the allocation for each price point to 0
        allocation_data[retailer] = {1:0, 2:0, 3:0, 5:0, 10:0, 20:0}
        
        # Add the allocated units for this retailer to the total for this price point
        allocation_data[retailer][Price] += allocated

# Now we have a dictionary that stores the allocation for each retailer at each price point
# Will iterate through the dictionary and update the allocation for each retailer at each price point

for retailer, allocation in allocation_data.items():
    for Price, allocated in allocation.items():
        if allocated == 0:
            # If the total allocations is 0, update the allocations for the first product with sell through < 60
            rows = report[(report['Retailer_id'] == retailer) & (report['Price_Point'] == Price) & (report['Sell_Through(%)'] < 60)]
            
            if not rows.empty:
                report.loc[rows.index[0],'Allocated'] += 1
                
                
report = report.set_index(['Retailer_id','Retailer_Name_x'])

with pd.ExcelWriter('K:/Output.xlsx') as writer:
    report.to_excel(writer,sheet_name ='Allocation',index = True)
    df1.style.set_properties(**{'text-align': 'center'}).to_excel(writer,sheet_name ='Stock', index = 0)

